# Week 2

### Context
#### Regression
+ Linear Regression
+ Ridge Linear Regression(L1)
+ Lasso Linear Regression(L2)

#### Evaluation
+ R<sup>2</sup>
+ MAE, MSE, RMSE

In [ ]:
import os
from os.path import join

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
BASE_DIR = ''

train_path = join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
x_test = pd.read_csv(test_path)

label = data['credit']

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
x_test.head()

In [ ]:
x_test.describe()

In [ ]:
x_test.info()

In [ ]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
x_test.drop(columns=['index'],         inplace=True)

In [ ]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

#### 1. 결측치 확인 및 결측치 처리 예시

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(data)

In [ ]:
msno.matrix(x_test)

##### 수치형 변수들의 결측치 확인

In [ ]:
pd.isna(data[num_columns]).sum()

In [ ]:
pd.isna(x_test[num_columns]).sum()

##### 범주형 변수들의 결측치 확인

In [ ]:
pd.isna(data[cat_columns]).sum()

In [ ]:
pd.isna(x_test[cat_columns]).sum()

##### 데이터 쪼개기, Train -> (Train, Valid)
- train_test_split 파라미터 
    - test_size  (float): Valid(test)의 크기의 비율을 지정
    - random_state (int): 데이터를 쪼갤 때 내부적으로 사용되는 난수 값 (해당 값을 지정하지 않으면 매번 달라집니다.)
    - shuffle     (bool): 데이터를 쪼갤 때 섞을지 유무
    - stratify   (array): Stratify란, 쪼개기 이전의 클래스 비율을 쪼개고 나서도 유지하기 위해 설정해야하는 값입니다. 클래스 라벨을 넣어주면 됩니다.
        - ex) 원본 Train 데이터의 클래스 비율이 (7:3) 이었다면, 쪼개어진 Train, Valid(test) 데이터의 클래스 비율도 (7:3)이 됩니다. 당연히 분류 데이터에서만 사용할 수 있습니다.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 쪼개어진 Train, Valid 데이터의 비율은 (7:3), 내부 난수 값 42, 데이터를 쪼갤 때 섞으며 label 값으로 Stratify 하는 코드 입니다. random_state를 주석 처리하고 데이터를 확인해보시면 계속 바뀝니다.
x_train, x_valid, y_train, y_valid = train_test_split(data, label, 
                                                      test_size=0.3,
                                                      random_state=42,
                                                      shuffle=True,
                                                      stratify=label)

In [ ]:
# 쪼갠 데이터의 인덱스는 정리해주는것이 좋습니다. pd.concat 연산 시, 인덱스를 기준으로 연결하기 때문입니다.
# drop 인자를 True로 주지 않으면 이전 인덱스가 새로운 변수로 생성됩니다.
x_train = x_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)

### 1) 전처리 실습 

#### 1. 결측치 처리
`occyp_type` 변수 최빈 값으로 결측치 처리

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
x_train[cat_columns] = imputer.fit_transform(x_train[cat_columns])
x_valid[cat_columns] = imputer.transform(x_valid[cat_columns])
x_test[cat_columns]  = imputer.transform(x_test[cat_columns])

#### 2. 스케일링

In [ ]:
x_train_mean = np.mean(x_train[num_columns], axis=0)
x_train_std  = np.std(x_train[num_columns], axis=0)

# Numpy 브로드캐스팅 기능을 활용해 x_train의 평균과 표준편차로 x_train, x_valid의 스케일링을 진행해줍니다.
x_train.loc[:, num_columns] = (x_train[num_columns] - x_train_mean) / (x_train_std + 1e-10)
x_valid.loc[:, num_columns] = (x_valid[num_columns] - x_train_mean) / (x_train_std + 1e-10)
x_test.loc[:, num_columns]  = (x_test[num_columns]  - x_train_mean) / (x_train_std + 1e-10)

#### 스케일링 확인

In [ ]:
x_train[num_columns].describe()

In [ ]:
x_valid[num_columns].describe()

In [ ]:
x_test[num_columns].describe()

#### 왜 x_valid와 x_test는 평균이 0, 표준편차가 1이 아닌가요? -> x_train의 평균과 표준편차를 사용했기 때문에 x_valid의 평균과 표준편차가 0, 1이 아닐 수 있습니다.

#### 3. 범주형 변수 OneHot Encoding, 라벨 변수 Label Encoding 
- 범주형 변수 OneHot Encoding 처리

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse=False)

ohe.fit(x_train[cat_columns])

In [ ]:
# new_x_train_cat = ohe.transform(x_train[cat_columns])
# new_x_valid_cat = ohe.transform(x_valid[cat_columns])
# new_x_test_cat = ohe.transform(x_test[cat_columns])

In [ ]:
# 둘다 가능
x_all = pd.concat([x_train[cat_columns], x_valid[cat_columns], x_test[cat_columns]], axis=0)

# 전체에 대해 fiting
ohe.fit(x_all)

# ohe.categories_ 은 입력된 범주형 컬럼의 범주 값을 순서대로 담고 있습니다.
ohe_columns = ohe.categories_[0].tolist()
for col in ohe.categories_[1:]:
    ohe_columns += col.tolist()
ohe_columns

In [ ]:
new_x_train_cat = pd.DataFrame(ohe.transform(x_train[cat_columns]), columns=ohe_columns)
new_x_valid_cat = pd.DataFrame(ohe.transform(x_valid[cat_columns]), columns=ohe_columns)
new_x_test_cat  = pd.DataFrame(ohe.transform(x_test[cat_columns]),  columns=ohe_columns)

In [ ]:
# train set 개수 확인
new_x_train_cat.shape, x_train.shape

In [ ]:
# valid set 개수 확인
new_x_valid_cat.shape, x_valid.shape

In [ ]:
# test set 개수 확인
new_x_test_cat.shape, x_test.shape

In [ ]:
# 동일하게 데이터를 쪼갤 시 인덱스를 초기화합니다.
new_x_train_cat.reset_index(drop=True, inplace=True)
new_x_valid_cat.reset_index(drop=True, inplace=True)
new_x_test_cat.reset_index(drop=True,  inplace=True)

##### 기존 범주형 변수를 제거하고, Onehot Encoding된 변수를 추가합니다.

In [ ]:
# 기존 범주형 변수 제거
x_train.drop(columns=cat_columns, inplace=True)
x_valid.drop(columns=cat_columns, inplace=True)
x_test.drop(columns=cat_columns,  inplace=True)

In [ ]:
# Onehot Encoding 변수 추가
x_train = pd.concat([x_train[num_columns], new_x_train_cat], axis=1)
x_valid = pd.concat([x_valid[num_columns], new_x_valid_cat], axis=1)
x_test  = pd.concat([x_test[num_columns],  new_x_test_cat],  axis=1)

In [ ]:
# train 확인
x_train.head()

In [ ]:
# valid 확인
x_valid.head()

In [ ]:
# test 확인
x_test.head()

In [ ]:
y_train = y_train.astype(int)
y_valid = y_valid.astype(int)

## 주의!! 분류 라벨 데이터가지고 회귀 모델로 예측하는건 잘못된 겁니다.

## Linear Regression
선형 회귀는 종속 변수와 한개 이상의 독립 변수와의 선형 상관 관계를 모델링하는 회귀 분석 기법입니다. <br>
용어를 종속 변수, 독립 변수로 표현하면 이해하기 어려우니 다음 수식에서의 y, x 로 표현하겠습니다.<br> 

$$ y = wx + b$$
$$ y = w_0x_0 + w_1x_1 + w_2x_2 + .... w_nx_n + b$$
$$ w : 계수(가중치) $$
$$ b : 절편(편향) $$

간단하게 생각해보면 선형 회귀는 데이터가 분포되어 있는 공간에서 데이터를 가장 잘 표현하는 선을 하나 긋는다고 생각할 수 있습니다.<br>
선형 회귀의 비용 함수는 다음과 같이 표현될 수 있습니다.

$$ Cost_{lr} = \sum_i{(y_i - \hat y_i)^2}$$
$$ \hat y_i = b + wx_i $$

결국 실제 참값 $y_i$와 회귀 모델이 출력한 $ \hat y $ 사이의 잔차의 제곱의 합을 최소화하는 w(계수)를 구하는 것이 목적입니다. -> Least Square, 최소 제곱법 <br>
선형 회귀는 출력되는 y가 1개 또는 2개 이상인지의 유무에 따라 단변량, 다변량이라는 말이 붙는데, 이번 수업에서는 출력값인 y가 1개(단변량)라고 가정하겠습니다. <br>
또한, 입력으로 들어가는 x가 1개 또는 2개 이상인지의 유무에 따라 단순(Simple), 다중(Multiple)이라는 말이 붙는데, 이번 실습에서는 단순, 다중 선형 회귀 분석에 대해 모두 알아보겠습니다.

#### 선형 회귀분석의 4가지 기본 가정
선형 회귀에는 4가지 가정이 필요합니다. 우리 수업에서는 이론적인 내용을 다루지 않으므로, 추후에 살펴보시면 좋겠습니다.<br>
맨 아래 참조 목록에 4가지 가정에 대해 잘 설명해준 페이지의 링크를 달아두었습니다.
1. 선형성
2. 독립성
3. 등분산성
4. 정규성

### 1. Linear Regression

- 하나의 연속 변수를 예측하는 선형 회귀에는 "단순" 선형 회귀(Simple Linear Regression), "다중" 선형 회귀(Multiple Linear Regression)가 있습니다. "단순" 선형 회귀는 입력 데이터의 변수가 1개인 경우이고, "다중" 선형 회귀는 입력 데이터의 변수가 2개 이상인 경우 입니다.
- 일반적으로는 여러 변수를 사용하는 "다중" 회귀 분석이 되겠습니다.

Linear Regression은 Sklearn의 linear_model 패키지에 있습니다.

- Linear Regression 대표적 파라미터
    - fit_intercept (bool) : 회귀 수식에서 y 절편을 포함할지 유무
    
#### ref
- [선형 회귀의 기본 가정 kkokkilkon님 블로그](https://kkokkilkon.tistory.com/175)
- [Linear Regression, Wiki](https://ko.wikipedia.org/wiki/선형_회귀)
- [Scikit-learn, Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

#### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
lr.fit(x_train, y_train)

#### 3) 결과 예측하기 (검증 데이터)

In [ ]:
y_pred = lr.predict(x_valid)

#### 4) 결과 살펴보기
일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용합니다.<br>
R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미합니다.

In [ ]:
from sklearn.metrics import r2_score

print('선형 회귀, R2 : {:.4f}'.format(r2_score(y_valid, y_pred)))

#### 회귀 모델의 계수 w, 절편 b 살펴보기
어떤 변수에 얼마 만큼의 가중치가 할당되고, 절편 값은 얼마나 할당되는지 살펴볼 수 있습니다.

In [ ]:
print('다중 선형 회귀, 계수(w) : {}, 절편(b) : {:.4f}'.format(lr.coef_, lr.intercept_))

### 2. Lasso Linear Regression(L1)
이번에는 선형 회귀 모델의 가중치 크기에 규제를 적용하는 라쏘 선형 회귀 모델을 살펴보겠습니다.<br>
라쏘 선형 회귀 모델과 기존 선형 회귀 모델의 차이점으로는 손실 함수에 L1 정규화 항을 추가하는 것인데, 다음과 같습니다. 

$$ Cost_{lr} = \sum_i{(y_i - \hat y_i)^2} + \lambda \sum{|w|}$$

해당 식에서 w는 선형 회귀 모델이 가진 가중치 값으로, 학습을 통해 손실 함수를 최소화하면 각 가중치의 값도 작아지는 효과를 얻을 수 있습니다.<br>
이는 선형 회귀 모델의 과적합을 방지하는 요소로써 사용되어지는데, 가중치의 값이 클 경우 모델의 복잡도가 높아져서 과적합되는 경향이 있기 때문입니다. <br>
라쏘 선형 회귀는 몇몇 가중치의 값을 0으로 만드는 특징이 있습니다.

- Lasso Linear Regression 대표적 파라미터
    - alpha         (float): L1 규제를 얼마나 많이 적용할지에 대한 수치 
    - fit_intercept (bool) : 회귀 수식에서 y 절편을 포함할지 유무
    - random_state  (int)  : 내부적으로 사용되는 난수값

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.linear_model import Lasso
lasso_lr = Lasso(alpha=0.1)

#### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
lasso_lr.fit(x_train, y_train)

#### 3) 결과 예측하기 (검증 데이터)

In [ ]:
y_pred = lasso_lr.predict(x_valid)

#### 4) 결과 살펴보기
일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용합니다.<br>
R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미합니다.

In [ ]:
print('라쏘 선형 회귀, R2 : {:.4f}'.format(r2_score(y_valid, y_pred)))

#### 회귀 모델의 계수 w, 절편 b 살펴보기
어떤 변수에 얼마 만큼의 가중치가 할당되고, 절편 값은 얼마나 할당되는지 살펴볼 수 있습니다.

In [ ]:
print('라쏘 선형 회귀, 계수(w) : {}, 절편(b) : {:.4f}'.format(lasso_lr.coef_, lasso_lr.intercept_))

### 3. Ridge Linear Regression(L2)
이번에는 선형 회귀 모델의 가중치 크기에 규제를 적용하는 릿지 선형 회귀 모델을 살펴보겠습니다.<br>
릿지 선형 회귀 모델과 기존 선형 회귀 모델의 차이점으로는 손실 함수에 L2 정규화 항을 추가하는 것인데, 다음과 같습니다. 

$$ Cost_{lr} = \sum_i{(y_i - \hat y_i)^2} + \lambda \sum{w^2}$$

해당 식에서 w는 선형 회귀 모델이 가진 가중치 값으로, 학습을 통해 손실 함수를 최소화하면 각 가중치의 값도 작아지는 효과를 얻을 수 있습니다.<br>
라쏘 선형 회귀와는 다르게 각 가중치의 값이 0이 아닌 0에 가까운 작은 수로 수렴하게 됩니다.

- Lasso Linear Regression 대표적 파라미터
    - alpha         (float): L2 규제를 얼마나 많이 적용할지에 대한 수치 
    - fit_intercept (bool) : 회귀 수식에서 y 절편을 포함할지 유무
    - random_state  (int)  : 내부적으로 사용되는 난수값

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.linear_model import Ridge
ridge_lr = Ridge()

#### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
ridge_lr.fit(x_train, y_train)

#### 3) 결과 예측하기 (검증 데이터)

In [ ]:
y_pred = ridge_lr.predict(x_valid)

#### 4) 결과 살펴보기
일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용합니다.<br>
R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미합니다.

In [ ]:
print('릿지 선형 회귀, R2 : {:.4f}'.format(r2_score(y_valid, y_pred)))

#### 회귀 모델의 계수 w, 절편 b 살펴보기
어떤 변수에 얼마 만큼의 가중치가 할당되고, 절편 값은 얼마나 할당되는지 살펴볼 수 있습니다.

In [ ]:
print('릿지 선형 회귀, 계수(w) : {}, 절편(b) : {:.4f}'.format(ridge_lr.coef_, ridge_lr.intercept_))

## Evaluation
### R<sup>2</sup>
Scikit-Learn에서 지원하는 회귀 모델의 평가 방법으로는 R<sup>2</sup>가 있습니다. <br>
학습한 회귀 모델이 얼마나 데이터를 잘 표현하는지에 대한 정도를 나타내는 통계적인 척도이며, 0 < R<sup>2</sup> < 1 범위의 값을 갖습니다.<br>

<img src = './img/R2.png' alt='R2' align='left' height=500 width=500 /> 

$$ R^2 = 1 - {SSE \over SST} = {SSR \over SST} = {선형모형의\ 편차 \over 전체편차}$$

* R<sup>2</sup> = 1, 모델이 데이터를 완벽하게 표현함 (Fits perfectly)
* R<sup>2</sup> = 0, 모델이 데이터를 전혀 표현하지 못함 (Does not explain anything)

- $ Var={{1}\over{n}}{\sum^n_{i=1}{(y_i - \bar{y})^2}} $

- SST = 분산 공식에서 샘플의 개수로 나누지 않은 수식 ~= 분산 ~= 편차
- SSR = 회귀선이 그리는 지점에서 평균을 뺀것이기 때문에, 회귀선이 나타내는 분산 ~= 편차
- 따라서 $R^2$는 전체 편차에 대한 회귀 식의 편차 비율

### Mean Absolute Error(MAE)
MAE는 회귀 모델에서 자주 사용되는 손실 함수 입니다. <br>
MAE는 오차(정답과 예측값의 차이)의 절대 값에 대해 평균을 취한 결과 입니다. 수식은 다음과 같습니다.
$${{1}\over{n}} \sum_{i=1}^n{(|y_i - \hat{y}_i|)}$$

### Mean Squared Error(MSE)
MSE는 회귀 모델에서 자주 사용되는 손실 함수로 다음과 같은 수식으로 구성되어 있습니다. <br>
이름에서 알 수 있듯 MAE가 오차의 절대값을 사용했다면, 오차를 제곱해서 평균을 취한 결과입니다. 
$${{1}\over{n}} \sum_{i=1}^n{(y_i - \hat{y}_i)^2}$$

### Root Mean Squared Error(RMSE)
RMSE는 MSE에 Square root를 씌운것과 같습니다. 수식은 다음과 같습니다. <br>

$$\sqrt{{{1}\over{n}} \sum_{i=1}^n{(y_i - \hat{y}_i)^2}}$$

#### ref 
- [MAE, MSE](https://blog.naver.com/PostView.nhn?blogId=heygun&logNo=221516529668&parentCategoryNo=&categoryNo=56&viewDate=&isShowPopularPosts=true&from=search)
- [RMSE](https://ko.wikipedia.org/wiki/평균_제곱근_편차)
- [$R^2$](https://m.blog.naver.com/pmw9440/221822183325)
- [Scikit-learn, MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html?highlight=mean%20absolute%20error#sklearn.metrics.mean_absolute_error)
- [Scikit-learn, MSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html?highlight=mean%20squared%20error#sklearn.metrics.mean_squared_error)
- [Scikit-learn, $R^2$](https://www.google.com/url?q=http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html&sa=U&ved=0ahUKEwicgu3vueDhAhUI9LwKHeLDD3UQFggEMAA&client=internal-uds-cse&cx=016639176250731907682:tjtqbvtvij0&usg=AOvVaw3JYUuCpR-KNsPU189XgvWR)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
print('r2_score : {:.3f}'.format(r2_score(y_valid, y_pred)))
print('MAE      : {:.6f}'.format(mean_absolute_error(y_valid, y_pred)))
print('MSE      : {:.6f}'.format(mean_squared_error(y_valid, y_pred)))
print('RMSE     : {:.6f}'.format(np.sqrt(mean_squared_error(y_valid, y_pred))))

## 실습 솔루션

### 1) 전처리 실습 

#### 1. 결측치 확인 및 결측치 처리 예시

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(data)

In [ ]:
msno.matrix(x_test)

##### 수치형 변수들의 결측치 확인

In [ ]:
pd.isna(data[num_columns]).sum()

In [ ]:
pd.isna(x_test[num_columns]).sum()

##### 범주형 변수들의 결측치 확인

In [ ]:
pd.isna(data[cat_columns]).sum()

In [ ]:
pd.isna(x_test[cat_columns]).sum()

결측치가 없으므로 그대로 넘어가셔도 됩니다.

##### 데이터 쪼개기, Train -> (Train, Valid)
- train_test_split 파라미터 
    - test_size  (float): Valid(test)의 크기의 비율을 지정
    - random_state (int): 데이터를 쪼갤 때 내부적으로 사용되는 난수 값 (해당 값을 지정하지 않으면 매번 달라집니다.)
    - shuffle     (bool): 데이터를 쪼갤 때 섞을지 유무
    - stratify   (array): Stratify란, 쪼개기 이전의 클래스 비율을 쪼개고 나서도 유지하기 위해 설정해야하는 값입니다. 클래스 라벨을 넣어주면 됩니다.
        - ex) 원본 Train 데이터의 클래스 비율이 (7:3) 이었다면, 쪼개어진 Train, Valid(test) 데이터의 클래스 비율도 (7:3)이 됩니다. 당연히 분류 데이터에서만 사용할 수 있습니다.
        - 이번에는 분류 문제가 아니므로 일반적으로는 stratify를 사용할 수 없습니다.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 쪼개어진 Train, Valid 데이터의 비율은 (7:3), 내부 난수 값 42, 데이터를 쪼갤 때 섞으며 label 값으로 Stratify 하는 코드 입니다. random_state를 주석 처리하고 데이터를 확인해보시면 계속 바뀝니다.
x_train, x_valid, y_train, y_valid = train_test_split(data, label, 
                                                      test_size=0.3,
                                                      random_state=42,
                                                      shuffle=True)

In [ ]:
# 쪼갠 데이터의 인덱스는 정리해주는것이 좋습니다. pd.concat 연산 시, 인덱스를 기준으로 연결하기 때문입니다.
# drop 인자를 True로 주지 않으면 이전 인덱스가 새로운 변수로 생성됩니다.
x_train.reset_index(drop=True, inplace=True)
x_valid.reset_index(drop=True, inplace=True)

##### x_train, x_valid, x_test에 대해 스케일링 및 범주형 변수 인코딩을 진행해보세요.

#### 2. 스케일링

In [ ]:
x_train_mean = np.mean(x_train[num_columns], axis=0)
x_train_std  = np.std(x_train[num_columns], axis=0)

# Numpy 브로드캐스팅 기능을 활용해 x_train의 평균과 표준편차로 x_train, x_valid의 스케일링을 진행해줍니다.
x_train.loc[:, num_columns] = (x_train[num_columns] - x_train_mean) / (x_train_std + 1e-10)
x_valid.loc[:, num_columns] = (x_valid[num_columns] - x_train_mean) / (x_train_std + 1e-10)
x_test.loc[:, num_columns]  = (x_test[num_columns]  - x_train_mean) / (x_train_std + 1e-10)

#### 스케일링 확인

In [ ]:
x_train[num_columns].describe()

In [ ]:
x_valid[num_columns].describe()

In [ ]:
x_test[num_columns].describe()

#### 왜 x_valid와 x_test는 평균이 0, 표준편차가 1이 아닌가요? -> x_train의 평균과 표준편차를 사용했기 때문에 x_valid의 평균과 표준편차가 0, 1이 아닐 수 있습니다.

#### 3. 범주형 변수 OneHot Encoding, 라벨 변수 스케일링
- 범주형 변수 OneHot Encoding 처리, 이번에는 라벨이 수치형이므로 라벨은 인코딩하지 않습니다.

#### Tip
- 예측할 변수의 값이 너무 큰 경우 라벨 변수를 log 스케일링을 진행할 수 있습니다. 거래가는 모두 양수이므로 log를 취해 예측할 라벨의 범위를 줄일 수 있습니다.
    - 예측 후, exp를 취해 원래 스케일로 되돌려줄 수 있습니다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(sparse=False)

ohe.fit(data[cat_columns])

In [ ]:
ohe_columns = ohe.categories_[0].tolist()
for cols in ohe.categories_[1:]:
    ohe_columns += cols.tolist()

In [ ]:
new_x_train_cat = pd.DataFrame(ohe.transform(x_train[cat_columns]), columns=ohe_columns)
new_x_valid_cat = pd.DataFrame(ohe.transform(x_valid[cat_columns]), columns=ohe_columns)
new_x_test_cat  = pd.DataFrame(ohe.transform(x_test[cat_columns]),  columns=ohe_columns)

In [ ]:
# train set 개수 확인
new_x_train_cat.shape, x_train.shape

In [ ]:
# valid set 개수 확인
new_x_valid_cat.shape, x_valid.shape

In [ ]:
# test set 개수 확인
new_x_test_cat.shape, x_test.shape

##### 기존 범주형 변수를 제거하고, Onehot Encoding된 변수를 추가합니다.

In [ ]:
# 기존 범주형 변수 제거
x_train.drop(columns=cat_columns, inplace=True)
x_valid.drop(columns=cat_columns, inplace=True)
x_test.drop(columns=cat_columns, inplace=True)

In [ ]:
# Onehot Encoding 변수 추가
x_train = pd.concat([x_train, new_x_train_cat], axis=1)
x_valid = pd.concat([x_valid, new_x_valid_cat], axis=1)
x_test = pd.concat([x_test, new_x_test_cat], axis=1)

In [ ]:
# train 확인
x_train.head()

In [ ]:
# valid 확인
x_valid.head()

In [ ]:
# test 확인
x_test.head()

In [ ]:
# 라벨값 스케일링
y_train = np.log(y_train)
y_valid = np.log(y_valid)

### 2) 모델 실습

#### XGBoost

##### 1) 모델 불러오기 및 정의하기

In [ ]:
from xgboost import XGBRegressor
xgb_regr = XGBRegressor()

##### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
xgb_regr.fit(x_train, y_train)

##### 3) 결과 예측하기 (검증 데이터)

In [ ]:
y_pred = xgb_regr.predict(x_valid)

##### 4) 결과 살펴보기
일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용합니다.<br>
R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미합니다.

In [ ]:
print('XGBoost 회귀, R2 : {:.4f}'.format(r2_score(y_valid, y_pred)))

#### LightGBM

##### 1) 모델 불러오기 및 정의하기

In [ ]:
from lightgbm import LGBMRegressor
lgb_regr = LGBMRegressor()

##### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
lgb_regr.fit(x_train, y_train)

##### 3) 결과 예측하기 (검증 데이터)

In [ ]:
y_pred = lgb_regr.predict(x_valid)

##### 4) 결과 살펴보기
일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용합니다.<br>
R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미합니다.

In [ ]:
print('LightGBM 회귀, R2 : {:.4f}'.format(r2_score(y_valid, y_pred)))

#### 평가 지표

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
# xgboost ver
regr = XGBRegressor(n_estimators=1000,
                    max_depth=8)

regr.fit(x_train, y_train)

y_pred = regr.predict(x_valid)

In [ ]:
print('r2_score : {:.3f}'.format(r2_score(y_valid, y_pred)))
print('MAE      : {:.6f}'.format(mean_absolute_error(y_valid, y_pred)))
print('MSE      : {:.6f}'.format(mean_squared_error(y_valid, y_pred)))
print('RMSE     : {:.6f}'.format(np.sqrt(mean_squared_error(y_valid, y_pred))))